In [2]:
# -*- coding: utf-8 -*-
 
import numpy as np
 
def Apriori_gen(Itemset, tamano):
    """Generar nuevos (k+1)-itemsets"""
    candidatos = []
    for i in range(0, tamano):
        element = Itemset[i]
        for j in range(i + 1, tamano):
            element1 = Itemset[j]
            if element[:-1] == element1[:-1]:
                unionset = element + element1[-1]
                candidatos.append(unionset)
    return candidatos
 
def Apriori_prune(Ck,MinSoport):
    L = []
    for i in Ck:
        if Ck[i] >= minsoport:
            L.append(i)
    return sorted(L)
 
def Apriori_conteo_subset(Candidato,Candidato_tamano):
    Lk = dict()
    archivo = open('datos3.txt')
    for l in archivo:
        l = str(l.split())
        conteo = 0
        for i in range (0,Candidato_tamano):
            habilitar = str(Candidato[i])
            if habilitar not in Lk:
                Lk[habilitar] = 0
            bandera = True
            for k in habilitar:
                if k not in l:
                    bandera = False
            if bandera:
                Lk[habilitar] += 1
    archivo.close()
    return Lk
 
minsoport = 2
C1={}
archivo = open('datos3.txt')
 
for linea in archivo:
    for item in linea.split():
        if item in C1:
            C1[item] +=1
        else:
            C1[item] = 1
archivo.close()
print(C1)
L = []
L1 = Apriori_prune(C1,minsoport)
L = Apriori_gen(L1,len(L1))
print ('*************************************')
print ('Frecuencia 1-itemset ',L1)
print ('*************************************')
k=2
while L != []:
    C = dict()
    C = Apriori_conteo_subset(L,len(L))
    frecuencia_itemset = []
    frecuencia_itemset = Apriori_prune(C,minsoport)
    print ('----------------------------------')
    print ('Frecuencia',k,'-itemset ',frecuencia_itemset)
    print ('----------------------------------')
    L = Apriori_gen(frecuencia_itemset,len(frecuencia_itemset))
    k += 1

{'I1': 4, 'I2': 5, 'I3': 4, 'I4': 4, 'I5': 2}
*************************************
Frecuencia 1-itemset  ['I1', 'I2', 'I3', 'I4', 'I5']
*************************************
----------------------------------
Frecuencia 2 -itemset  ['I12', 'I13', 'I14', 'I23', 'I24', 'I34']
----------------------------------
----------------------------------
Frecuencia 3 -itemset  ['I123', 'I124', 'I234']
----------------------------------
----------------------------------
Frecuencia 4 -itemset  []
----------------------------------


**FP**

In [135]:
#Función para cargar archivos y devolver listas de transacciones
def cargar_datos(archivo):
    with open(archivo) as f:
        contenido = f.readlines()

    contenido = [x.strip() for x in contenido]
    Transaccion = []

    for i in range(0, len(contenido)):
        Transaccion.append(contenido[i].split())

    return Transaccion

#Para convertir la transacción inicial y mantenerla frozenset
def Set_inicio(dataset):
    retDict = {}
    for trans in dataset:
        retDict[frozenset(trans)] = 1
    return retDict

#clase de nodo arbol FP
class nodos_arbol:
    def __init__(self, Nodo,conteo,nodoPadre):
        self.nodoC = Nodo
        self.count = conteo
        self.nodoLiga = None
        self.padre = nodoPadre
        self.hijo = {}

    def incremento_contador(self, contador):
        self.count += contador

#Para crear Raiz_arbol y itemsets ordenados para arbol FP
def crear_FP_Arbol(dataset, minSoport):
    Raiz_arbol = {}
    for transaccion in dataset:
        for item in transaccion:
            Raiz_arbol[item] = Raiz_arbol.get(item,0) + dataset[transaccion]
    for k in list(Raiz_arbol):
        if Raiz_arbol[k] < minSoport:
            del(Raiz_arbol[k])

    frecuent_itemset = set(Raiz_arbol.keys())

    if len(frecuent_itemset) == 0:
        return None, None

    for k in Raiz_arbol:
        Raiz_arbol[k] = [Raiz_arbol[k], None]

    retTree = nodos_arbol('Null Set',1,None)
    for itemset,contar in dataset.items():
        frecuen_transaccion = {}
        for item in itemset:
            if item in frecuent_itemset:
                frecuen_transaccion[item] = Raiz_arbol[item][0]
        if len(frecuen_transaccion) > 0:
            ordenar_itemset = [v[0] for v in sorted(frecuen_transaccion.items(), key=lambda p: p[1], reverse=True)]
            #actualizar FPTree
            actualizarArbol(ordenar_itemset, retTree, Raiz_arbol, contar)
    return retTree, Raiz_arbol

def actualizarArbol(itemset, FPTree, Raiz_arbol, count):
    if itemset[0] in FPTree.hijo:
        FPTree.hijo[itemset[0]].incremento_contador(count)
    else:
        FPTree.hijo[itemset[0]] = nodos_arbol(itemset[0], count, FPTree)

        if Raiz_arbol[itemset[0]][1] == None:
            Raiz_arbol[itemset[0]][1] = FPTree.hijo[itemset[0]]
        else:
            actualizar_nodo(Raiz_arbol[itemset[0]][1], FPTree.hijo[itemset[0]])

    if len(itemset) > 1:
        actualizarArbol(itemset[1::], FPTree.hijo[itemset[0]], Raiz_arbol, count)


def actualizar_nodo(Nodo_test, nodo_objetivo):
    while (Nodo_test.nodoLiga != None):
        Nodo_test = Nodo_test.nodoLiga

    Nodo_test.nodoLiga = nodo_objetivo


def FPTree_transversal(nodo_hoja, ruta_opcion):
 if nodo_hoja.padre != None:
    ruta_opcion.append(nodo_hoja.nodoC)
    FPTree_transversal(nodo_hoja.padre, ruta_opcion)

def encontrar_ruta(basePat, nodos_arbol):
 patron_cond_base = {}

 while nodos_arbol != None:
    ruta_opcion = []
    FPTree_transversal(nodos_arbol, ruta_opcion)
    if len(ruta_opcion) > 1:
        patron_cond_base[frozenset(ruta_opcion[1:])] = nodos_arbol.count
    nodos_arbol = nodos_arbol.nodoLiga

 return patron_cond_base


def Buscar_Tree(FPTree, indicarTabla, minSuport, prefix, frecuent_itemset):
    bigL = [v[0] for v in sorted(indicarTabla.items(),key=lambda p: p[1][0])]
    for basePat in bigL:
        new_frecuentset = prefix.copy()
        new_frecuentset.add(basePat)

        frecuent_itemset.append(new_frecuentset)

        Patron_base_condi = encontrar_ruta(basePat, indicarTabla[basePat][1])

        Condicional_FPTree, Cond_cabecera = crear_FP_Arbol(Patron_base_condi,minSuport)

        if Cond_cabecera != None:
            Buscar_Tree(Condicional_FPTree, Cond_cabecera, minSuport, new_frecuentset, frecuent_itemset)

def cargar_datos_csv(archivo):
    Transaccion = []

    with open(archivo, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            transaccion = [elemento.strip() for elemento in row]
            Transaccion.append(transaccion)

    return Transaccion

import csv

archivo = 'datos3.txt'
print("Mínimo conteo de soporte:")
soporte_minimo = 1
print(soporte_minimo)
initSet = Set_inicio(cargar_datos(archivo))
FPtree, indicarTabla = crear_FP_Arbol(initSet, soporte_minimo)

frecuent_itemset = []

Buscar_Tree(FPtree, indicarTabla, soporte_minimo, set([]), frecuent_itemset)
frecuent_itemset = [itemset for itemset in frecuent_itemset if not any(element == '' for element in itemset)]

print("Todas las frecuencias de los itemsets:")
print(frecuent_itemset)

Mínimo conteo de soporte:
1
Todas las frecuencias de los itemsets:
[{'I5'}, {'I5', 'I4'}, {'I5', 'I3'}, {'I2', 'I5'}, {'I2', 'I5', 'I3'}, {'I5', 'I1'}, {'I2', 'I5', 'I1'}, {'I3', 'I5', 'I1'}, {'I3', 'I2', 'I5', 'I1'}, {'I3'}, {'I2', 'I3'}, {'I1'}, {'I3', 'I1'}, {'I3', 'I2', 'I1'}, {'I2', 'I1'}, {'I4'}, {'I3', 'I4'}, {'I2', 'I3', 'I4'}, {'I1', 'I4'}, {'I3', 'I1', 'I4'}, {'I3', 'I2', 'I1', 'I4'}, {'I2', 'I1', 'I4'}, {'I2', 'I4'}, {'I2'}]


{BREAD} -> {} (conf: 0.615, supp: 0.400, lift: 1.026, conv: 1.040)
{} -> {BREAD} (conf: 0.667, supp: 0.400, lift: 1.026, conv: 1.050)
{COFFEE} -> {} (conf: 0.625, supp: 0.250, lift: 1.042, conv: 1.067)
{} -> {COFFEE} (conf: 0.417, supp: 0.250, lift: 1.042, conv: 1.029)
{BREAD} -> {BISCUIT} (conf: 0.308, supp: 0.200, lift: 0.879, conv: 0.939)
{BISCUIT} -> {BREAD} (conf: 0.571, supp: 0.200, lift: 0.879, conv: 0.817)
{MILK} -> {BREAD} (conf: 0.800, supp: 0.200, lift: 1.231, conv: 1.750)
{BREAD} -> {MILK} (conf: 0.308, supp: 0.200, lift: 1.231, conv: 1.083)
{TEA} -> {BREAD} (conf: 0.571, supp: 0.200, lift: 0.879, conv: 0.817)
{BREAD} -> {TEA} (conf: 0.308, supp: 0.200, lift: 0.879, conv: 0.939)
{CORNFLAKES} -> {COFFEE} (conf: 0.667, supp: 0.200, lift: 1.667, conv: 1.800)
{COFFEE} -> {CORNFLAKES} (conf: 0.500, supp: 0.200, lift: 1.667, conv: 1.400)
{TEA} -> {JUICE} (conf: 0.571, supp: 0.200, lift: 2.286, conv: 1.750)
{JUICE} -> {TEA} (conf: 0.800, supp: 0.200, lift: 2.286, conv: 3.250)


In [179]:
from efficient_apriori import apriori
import csv

def cargar_datos_csv(archivo):
    Transaccion = []

    with open(archivo, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            transaccion = [elemento.strip() for elemento in row]
            Transaccion.append(transaccion)

    return Transaccion

archivo = 'grocery.csv'
archivo = cargar_datos_csv(archivo)
conjuntos, reglas = apriori(archivo, min_support=0.1, min_confidence=0.1)

for regla in reglas:
    print(regla)
    


{BISCUIT} -> {} (conf: 0.429, supp: 0.150, lift: 0.714, conv: 0.700)
{} -> {BISCUIT} (conf: 0.250, supp: 0.150, lift: 0.714, conv: 0.867)
{BOURNVITA} -> {} (conf: 0.750, supp: 0.150, lift: 1.250, conv: 1.600)
{} -> {BOURNVITA} (conf: 0.250, supp: 0.150, lift: 1.250, conv: 1.067)
{BREAD} -> {} (conf: 0.615, supp: 0.400, lift: 1.026, conv: 1.040)
{} -> {BREAD} (conf: 0.667, supp: 0.400, lift: 1.026, conv: 1.050)
{COFFEE} -> {} (conf: 0.625, supp: 0.250, lift: 1.042, conv: 1.067)
{} -> {COFFEE} (conf: 0.417, supp: 0.250, lift: 1.042, conv: 1.029)
{CORNFLAKES} -> {} (conf: 0.333, supp: 0.100, lift: 0.556, conv: 0.600)
{} -> {CORNFLAKES} (conf: 0.167, supp: 0.100, lift: 0.556, conv: 0.840)
{JUICE} -> {} (conf: 0.400, supp: 0.100, lift: 0.667, conv: 0.667)
{} -> {JUICE} (conf: 0.167, supp: 0.100, lift: 0.667, conv: 0.900)
{MILK} -> {} (conf: 0.400, supp: 0.100, lift: 0.667, conv: 0.667)
{} -> {MILK} (conf: 0.167, supp: 0.100, lift: 0.667, conv: 0.900)
{SPLENDA} -> {} (conf: 1.000, supp: 0.10